# GeoMatchAI Model Analysis & Visualization

This notebook analyzes model performance and creates beautiful visualizations.

**Prerequisites:** Run `usertest_comprehensive.py` first to generate test data.

## 0. Install Dependencies

Run this cell first to install required packages.

In [ ]:
import subprocess
import sys

packages = ['pandas', 'numpy', 'matplotlib', 'seaborn']

for package in packages:
    try:
        __import__(package)
        print(f"✓ {package} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '-q'])
        print(f"✓ {package} installed")

print("\nAll dependencies ready!")

## 1. Setup & Imports

In [ ]:
import os
import sys
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# Style settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

In [ ]:
# Paths
PROJECT_ROOT = Path().absolute().parent
OUTPUT_DIR = Path('output/csv')
IMG_DIR = PROJECT_ROOT / 'img'
IMG_DIR.mkdir(exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Output dir: {OUTPUT_DIR}")
print(f"Image dir: {IMG_DIR}")

## 2. Load Test Results

In [ ]:
try:
    df_summary = pd.read_csv(OUTPUT_DIR / 'results_summary.csv')
    df_by_image = pd.read_csv(OUTPUT_DIR / 'results_by_image.csv')
    df_by_model = pd.read_csv(OUTPUT_DIR / 'results_by_model.csv')
    df_by_landmark = pd.read_csv(OUTPUT_DIR / 'results_by_landmark.csv')
    
    print("✓ All CSV files loaded successfully!")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("\nPlease run the comprehensive test first:")
    print("  uv run examples/usertest_comprehensive.py")
    raise

In [ ]:
print("Dataset sizes:")
print(f"  - Summary: {len(df_summary):,} rows")
print(f"  - By Image: {len(df_by_image):,} rows")
print(f"  - By Model: {len(df_by_model):,} rows")
print(f"  - By Landmark: {len(df_by_landmark):,} rows")

In [ ]:
# Preview data
df_summary.head()

## 3. Overall Statistics

In [ ]:
total_tests = len(df_summary)
total_correct = df_summary['is_correct'].sum()
overall_accuracy = (total_correct / total_tests) * 100

print("=" * 60)
print("OVERALL STATISTICS")
print("=" * 60)
print(f"Total tests: {total_tests:,}")
print(f"Correct predictions: {total_correct:,}")
print(f"Overall accuracy: {overall_accuracy:.2f}%")

In [ ]:
print(f"Unique models tested: {df_summary['model_name'].nunique()}")
print(f"Unique landmarks: {df_summary['landmark_name'].nunique()}")
print(f"Unique images: {df_summary['image_name'].nunique()}")

In [ ]:
print("Accuracy by preprocessing:")
for prep in [True, False]:
    subset = df_summary[df_summary['preprocessing'] == prep]
    acc = (subset['is_correct'].sum() / len(subset)) * 100
    print(f"  {'WITH' if prep else 'WITHOUT'} preprocessing: {acc:.2f}%")

## 4. Model Accuracy Rankings

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# With preprocessing
top_models_with = df_by_model[df_by_model['preprocessing'] == True].nlargest(15, 'accuracy')
ax1.barh(range(len(top_models_with)), top_models_with['accuracy'] * 100)
ax1.set_yticks(range(len(top_models_with)))
ax1.set_yticklabels(top_models_with['model_name'], fontsize=9)
ax1.set_xlabel('Accuracy (%)')
ax1.set_title('Top 15 Models (WITH Preprocessing)', fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
ax1.invert_yaxis()

for i, v in enumerate(top_models_with['accuracy'] * 100):
    ax1.text(v + 1, i, f'{v:.1f}%', va='center', fontsize=8)

# Without preprocessing
top_models_without = df_by_model[df_by_model['preprocessing'] == False].nlargest(15, 'accuracy')
ax2.barh(range(len(top_models_without)), top_models_without['accuracy'] * 100, color='coral')
ax2.set_yticks(range(len(top_models_without)))
ax2.set_yticklabels(top_models_without['model_name'], fontsize=9)
ax2.set_xlabel('Accuracy (%)')
ax2.set_title('Top 15 Models (WITHOUT Preprocessing)', fontweight='bold')
ax2.grid(axis='x', alpha=0.3)
ax2.invert_yaxis()

for i, v in enumerate(top_models_without['accuracy'] * 100):
    ax2.text(v + 1, i, f'{v:.1f}%', va='center', fontsize=8)

plt.tight_layout()
plt.savefig(IMG_DIR / 'model_accuracy_rankings.png', dpi=300, bbox_inches='tight')
print(f"✓ Saved: {IMG_DIR / 'model_accuracy_rankings.png'}")
plt.show()

## 5. Discrimination Gap Analysis

The **discrimination gap** measures how well a model separates related images from unrelated ones.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))

df_disc = df_by_model[df_by_model['preprocessing'] == True].nlargest(20, 'discrimination_gap')

y_pos = np.arange(len(df_disc))
related_scores = df_disc['avg_related_score'].values * 100
unrelated_scores = df_disc['avg_unrelated_score'].values * 100

ax.barh(y_pos, related_scores, label='Related Images', alpha=0.8, color='green')
ax.barh(y_pos, unrelated_scores, label='Unrelated Images', alpha=0.8, color='red')

ax.set_yticks(y_pos)
ax.set_yticklabels(df_disc['model_name'], fontsize=9)
ax.set_xlabel('Similarity Score (%)')
ax.set_title('Top 20 Models by Discrimination Gap (WITH Preprocessing)', fontweight='bold')
ax.legend(loc='lower right')
ax.grid(axis='x', alpha=0.3)
ax.invert_yaxis()

for i, (rel, unrel) in enumerate(zip(related_scores, unrelated_scores)):
    gap = rel - unrel
    ax.text(max(rel, unrel) + 2, i, f'Δ{gap:.1f}%', va='center', fontsize=8, fontweight='bold')

plt.tight_layout()
plt.savefig(IMG_DIR / 'discrimination_gap.png', dpi=300, bbox_inches='tight')
print(f"✓ Saved: {IMG_DIR / 'discrimination_gap.png'}")
plt.show()

## 6. Speed vs Accuracy Trade-off

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))

df_perf = df_by_model[df_by_model['preprocessing'] == True]

scatter = ax.scatter(
    df_perf['avg_inference_time_s'] * 1000,
    df_perf['accuracy'] * 100,
    s=200,
    alpha=0.6,
    c=df_perf['discrimination_gap'],
    cmap='viridis',
    edgecolors='black',
    linewidth=0.5
)

# Annotate top performers
top_acc = df_perf.nlargest(5, 'accuracy')
for _, row in top_acc.iterrows():
    ax.annotate(
        row['model_name'],
        (row['avg_inference_time_s'] * 1000, row['accuracy'] * 100),
        xytext=(5, 5),
        textcoords='offset points',
        fontsize=8,
        bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5)
    )

ax.set_xlabel('Inference Time (ms)')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Speed vs Accuracy Trade-off (WITH Preprocessing)', fontweight='bold')
ax.grid(True, alpha=0.3)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Discrimination Gap')

plt.tight_layout()
plt.savefig(IMG_DIR / 'speed_vs_accuracy.png', dpi=300, bbox_inches='tight')
print(f"✓ Saved: {IMG_DIR / 'speed_vs_accuracy.png'}")
plt.show()

## 7. Landmark Difficulty Analysis

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Related landmarks accuracy
related_landmarks = df_by_landmark[
    (df_by_landmark['is_related'] == True) & 
    (df_by_landmark['preprocessing'] == True)
].sort_values('accuracy', ascending=True)

if len(related_landmarks) > 0:
    ax1.barh(range(len(related_landmarks)), related_landmarks['accuracy'] * 100, color='forestgreen')
    ax1.set_yticks(range(len(related_landmarks)))
    ax1.set_yticklabels(related_landmarks['landmark_name'])
    ax1.set_xlabel('Accuracy (%)')
    ax1.set_title('Landmark Recognition Accuracy\n(Related Images)', fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)
    
    for i, v in enumerate(related_landmarks['accuracy'] * 100):
        ax1.text(v + 1, i, f'{v:.1f}%', va='center', fontsize=9)

# Score distribution by landmark
landmark_scores = df_summary[
    (df_summary['preprocessing'] == True) & 
    (df_summary['is_related'] == True)
].groupby('landmark_name')['similarity_score'].apply(list)

if len(landmark_scores) > 0:
    ax2.boxplot(landmark_scores.values, labels=landmark_scores.index, vert=False)
    ax2.set_xlabel('Similarity Score')
    ax2.set_title('Score Distribution by Landmark', fontweight='bold')
    ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(IMG_DIR / 'landmark_analysis.png', dpi=300, bbox_inches='tight')
print(f"✓ Saved: {IMG_DIR / 'landmark_analysis.png'}")
plt.show()

## 8. Model Performance Heatmaps

In [ ]:
top_4_models = df_by_model[df_by_model['preprocessing'] == True].nlargest(4, 'accuracy')['model_name'].values
print(f"Top 4 models: {list(top_4_models)}")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, model_name in enumerate(top_4_models):
    model_data = df_summary[
        (df_summary['model_name'] == model_name) & 
        (df_summary['preprocessing'] == True)
    ]
    
    pivot = model_data.pivot_table(
        index='landmark_name',
        columns='is_related',
        values='similarity_score',
        aggfunc='mean'
    )
    
    sns.heatmap(
        pivot,
        annot=True,
        fmt='.3f',
        cmap='RdYlGn',
        center=0.65,
        vmin=0,
        vmax=1,
        ax=axes[idx],
        cbar_kws={'label': 'Similarity Score'}
    )
    
    axes[idx].set_title(f'{model_name}', fontweight='bold')
    axes[idx].set_xlabel('Is Related')
    axes[idx].set_ylabel('Landmark')

plt.tight_layout()
plt.savefig(IMG_DIR / 'model_heatmaps.png', dpi=300, bbox_inches='tight')
print(f"✓ Saved: {IMG_DIR / 'model_heatmaps.png'}")
plt.show()

## 9. Preprocessing Impact Analysis

In [ ]:
# Calculate preprocessing delta for each model
preprocessing_impact = []

for model_name in df_by_model['model_name'].unique():
    with_prep = df_by_model[
        (df_by_model['model_name'] == model_name) & 
        (df_by_model['preprocessing'] == True)
    ]['accuracy'].values
    
    without_prep = df_by_model[
        (df_by_model['model_name'] == model_name) & 
        (df_by_model['preprocessing'] == False)
    ]['accuracy'].values
    
    if len(with_prep) > 0 and len(without_prep) > 0:
        delta = (with_prep[0] - without_prep[0]) * 100
        preprocessing_impact.append({
            'model': model_name,
            'delta': delta,
            'with': with_prep[0] * 100,
            'without': without_prep[0] * 100
        })

df_impact = pd.DataFrame(preprocessing_impact).sort_values('delta', ascending=False)
df_impact.head(10)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Impact by model
top_15 = df_impact.head(15)
colors = ['green' if x > 0 else 'red' for x in top_15['delta']]
ax1.barh(range(len(top_15)), top_15['delta'], color=colors, alpha=0.7)
ax1.set_yticks(range(len(top_15)))
ax1.set_yticklabels(top_15['model'], fontsize=9)
ax1.set_xlabel('Accuracy Change (%)')
ax1.set_title('Preprocessing Impact (Positive = Better WITH)', fontweight='bold')
ax1.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax1.grid(axis='x', alpha=0.3)
ax1.invert_yaxis()

for i, v in enumerate(top_15['delta']):
    ax1.text(v + 0.2 if v > 0 else v - 0.2, i, f'{v:+.1f}%', 
             va='center', ha='left' if v > 0 else 'right', fontsize=8)

# Distribution comparison
with_prep_acc = df_by_model[df_by_model['preprocessing'] == True]['accuracy'] * 100
without_prep_acc = df_by_model[df_by_model['preprocessing'] == False]['accuracy'] * 100

ax2.hist([with_prep_acc, without_prep_acc], bins=20, 
         label=['WITH Preprocessing', 'WITHOUT Preprocessing'], 
         alpha=0.7, color=['green', 'red'])
ax2.set_xlabel('Accuracy (%)')
ax2.set_ylabel('Number of Models')
ax2.set_title('Accuracy Distribution', fontweight='bold')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(IMG_DIR / 'preprocessing_impact.png', dpi=300, bbox_inches='tight')
print(f"✓ Saved: {IMG_DIR / 'preprocessing_impact.png'}")
plt.show()

## 10. Architecture Family Comparison

In [ ]:
def categorize_model(model_name):
    """Categorize model by architecture family."""
    name_lower = model_name.lower()
    if 'vit' in name_lower or 'deit' in name_lower:
        return 'Vision Transformers'
    elif 'swin' in name_lower:
        return 'Swin Transformers'
    elif 'clip' in name_lower:
        return 'CLIP Models'
    elif 'efficientnet' in name_lower:
        return 'EfficientNet'
    elif 'convnext' in name_lower:
        return 'ConvNeXt'
    elif 'resnet' in name_lower or 'resnest' in name_lower:
        return 'ResNet Family'
    elif 'densenet' in name_lower:
        return 'DenseNet'
    elif 'regnet' in name_lower:
        return 'RegNet'
    elif 'nfnet' in name_lower:
        return 'NFNet'
    elif 'mobile' in name_lower:
        return 'MobileNet'
    elif 'inception' in name_lower:
        return 'Inception'
    else:
        return 'Other'

df_by_model['architecture'] = df_by_model['model_name'].apply(categorize_model)
df_by_model['architecture'].value_counts()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Stats by architecture
arch_stats = df_by_model[df_by_model['preprocessing'] == True].groupby('architecture').agg({
    'accuracy': ['mean', 'std'],
    'model_name': 'count'
}).sort_values(('accuracy', 'mean'), ascending=False)

arch_names = arch_stats.index
arch_means = arch_stats[('accuracy', 'mean')].values * 100
arch_stds = arch_stats[('accuracy', 'std')].values * 100
arch_counts = arch_stats[('model_name', 'count')].values

y_pos = np.arange(len(arch_names))
ax1.barh(y_pos, arch_means, xerr=arch_stds, alpha=0.7, capsize=5)
ax1.set_yticks(y_pos)
ax1.set_yticklabels([f"{name} (n={count})" for name, count in zip(arch_names, arch_counts)])
ax1.set_xlabel('Mean Accuracy (%) ± Std')
ax1.set_title('Architecture Family Comparison', fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
ax1.invert_yaxis()

for i, v in enumerate(arch_means):
    ax1.text(v + 2, i, f'{v:.1f}%', va='center', fontsize=9)

# Box plot
arch_data = []
arch_labels = []
for arch in arch_names:
    data = df_by_model[
        (df_by_model['architecture'] == arch) & 
        (df_by_model['preprocessing'] == True)
    ]['accuracy'].values * 100
    if len(data) > 0:
        arch_data.append(data)
        arch_labels.append(arch)

bp = ax2.boxplot(arch_data, labels=arch_labels, vert=False, patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
    patch.set_alpha(0.7)

ax2.set_xlabel('Accuracy (%)')
ax2.set_title('Performance Distribution', fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(IMG_DIR / 'architecture_comparison.png', dpi=300, bbox_inches='tight')
print(f"✓ Saved: {IMG_DIR / 'architecture_comparison.png'}")
plt.show()

## 11. Summary Dashboard

In [ ]:
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

In [ ]:
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Overall accuracy
ax1 = fig.add_subplot(gs[0, 0])
categories = ['Overall', 'WITH Prep', 'WITHOUT Prep']
accuracies = [
    overall_accuracy,
    (df_summary[df_summary['preprocessing'] == True]['is_correct'].sum() / 
     len(df_summary[df_summary['preprocessing'] == True])) * 100,
    (df_summary[df_summary['preprocessing'] == False]['is_correct'].sum() / 
     len(df_summary[df_summary['preprocessing'] == False])) * 100
]
ax1.bar(categories, accuracies, color=['blue', 'green', 'red'], alpha=0.7)
ax1.set_ylabel('Accuracy (%)')
ax1.set_title('Overall Performance', fontweight='bold')
ax1.set_ylim([0, 100])
for i, v in enumerate(accuracies):
    ax1.text(i, v + 2, f'{v:.1f}%', ha='center', fontweight='bold')

# 2. Best model stats
ax2 = fig.add_subplot(gs[0, 1])
best_model = df_by_model[df_by_model['preprocessing'] == True].nlargest(1, 'accuracy').iloc[0]
stats_text = f"""Best Model:
{best_model['model_name']}

Accuracy: {best_model['accuracy']*100:.2f}%
Discrimination: {best_model['discrimination_gap']:.3f}
Avg Related: {best_model['avg_related_score']:.3f}
Avg Unrelated: {best_model['avg_unrelated_score']:.3f}
Inference: {best_model['avg_inference_time_s']*1000:.1f}ms"""
ax2.text(0.1, 0.5, stats_text, fontsize=11, verticalalignment='center', 
         family='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax2.axis('off')
ax2.set_title('Champion Model', fontweight='bold')

# 3. Test distribution
ax3 = fig.add_subplot(gs[0, 2])
test_dist = df_summary['landmark_name'].value_counts()
ax3.pie(test_dist.values, labels=test_dist.index, autopct='%1.1f%%', startangle=90)
ax3.set_title('Tests by Landmark', fontweight='bold')

# 4. Top 10 models
ax4 = fig.add_subplot(gs[1, :])
top_10 = df_by_model[df_by_model['preprocessing'] == True].nlargest(10, 'accuracy')
x = np.arange(len(top_10))
width = 0.35
ax4.bar(x - width/2, top_10['accuracy'] * 100, width, label='Accuracy', alpha=0.8)
ax4.bar(x + width/2, top_10['discrimination_gap'] * 100, width, label='Discrim. Gap', alpha=0.8)
ax4.set_ylabel('Percentage')
ax4.set_title('Top 10 Models: Accuracy vs Discrimination Gap', fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(top_10['model_name'], rotation=45, ha='right', fontsize=9)
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

# 5. Inference time
ax5 = fig.add_subplot(gs[2, 0])
times = df_by_model[df_by_model['preprocessing'] == True]['avg_inference_time_s'] * 1000
ax5.hist(times, bins=20, alpha=0.7, color='purple', edgecolor='black')
ax5.axvline(times.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {times.mean():.1f}ms')
ax5.set_xlabel('Inference Time (ms)')
ax5.set_ylabel('Count')
ax5.set_title('Inference Time Distribution', fontweight='bold')
ax5.legend()

# 6. Score distribution
ax6 = fig.add_subplot(gs[2, 1])
related = df_summary[(df_summary['is_related'] == True) & (df_summary['preprocessing'] == True)]['similarity_score']
unrelated = df_summary[(df_summary['is_related'] == False) & (df_summary['preprocessing'] == True)]['similarity_score']
ax6.hist([related, unrelated], bins=30, alpha=0.7, label=['Related', 'Unrelated'], color=['green', 'red'])
ax6.axvline(0.65, color='black', linestyle='--', linewidth=2, label='Threshold (0.65)')
ax6.set_xlabel('Similarity Score')
ax6.set_ylabel('Count')
ax6.set_title('Score Distribution', fontweight='bold')
ax6.legend()

# 7. Model types
ax7 = fig.add_subplot(gs[2, 2])
arch_counts = df_by_model[df_by_model['preprocessing'] == True]['architecture'].value_counts()
ax7.pie(arch_counts.values, labels=arch_counts.index, autopct='%1.0f%%', startangle=90)
ax7.set_title('Models by Architecture', fontweight='bold')

fig.suptitle('GeoMatchAI Comprehensive Test Results', fontsize=18, fontweight='bold', y=0.995)

plt.savefig(IMG_DIR / 'summary_dashboard.png', dpi=300, bbox_inches='tight')
print(f"✓ Saved: {IMG_DIR / 'summary_dashboard.png'}")
plt.show()

## 12. Export Text Report

In [ ]:
lines = []
lines.append("=" * 80)
lines.append("GeoMatchAI Model Analysis Report")
lines.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
lines.append("=" * 80)
lines.append("")

lines.append("OVERALL STATISTICS")
lines.append("-" * 80)
lines.append(f"Total tests: {len(df_summary):,}")
lines.append(f"Overall accuracy: {overall_accuracy:.2f}%")
lines.append(f"Models tested: {df_summary['model_name'].nunique()}")
lines.append(f"Landmarks tested: {df_summary['landmark_name'].nunique()}")
lines.append("")

In [ ]:
lines.append("TOP 10 MODELS (WITH PREPROCESSING)")
lines.append("-" * 80)
top_10_w = df_by_model[df_by_model['preprocessing'] == True].nlargest(10, 'accuracy')
for i, (_, row) in enumerate(top_10_w.iterrows(), 1):
    lines.append(f"{i:2d}. {row['model_name']:<35} Acc: {row['accuracy']*100:5.2f}%  Gap: {row['discrimination_gap']:.3f}")
lines.append("")

In [ ]:
lines.append("TOP 10 MODELS (WITHOUT PREPROCESSING)")
lines.append("-" * 80)
top_10_wo = df_by_model[df_by_model['preprocessing'] == False].nlargest(10, 'accuracy')
for i, (_, row) in enumerate(top_10_wo.iterrows(), 1):
    lines.append(f"{i:2d}. {row['model_name']:<35} Acc: {row['accuracy']*100:5.2f}%  Gap: {row['discrimination_gap']:.3f}")
lines.append("")

In [ ]:
lines.append("IMAGES GENERATED")
lines.append("-" * 80)
for img in ['model_accuracy_rankings.png', 'discrimination_gap.png', 'speed_vs_accuracy.png',
            'landmark_analysis.png', 'model_heatmaps.png', 'preprocessing_impact.png',
            'architecture_comparison.png', 'summary_dashboard.png']:
    lines.append(f"  ✓ {img}")
lines.append("")
lines.append("=" * 80)

report = "\n".join(lines)
(IMG_DIR / 'analysis_report.txt').write_text(report)
print(report)
print(f"\n✓ Saved: {IMG_DIR / 'analysis_report.txt'}")